In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import core
import keras
import core.keras_models.RegressionTransformer as RegressionTransformer
import core.utils as utils

PLOTS_DIR = f"plots/regression_transformer/"
MODEL_DIR = f"models/regression_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], max_events=4_000_000, event_numbers="even"
)

X_train, y_train = DataProcessor.get_data()
del DataProcessor  # Free memory

2026-02-05 13:28:12.074793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770294492.096934  800385 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770294492.104542  800385 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770294492.122104  800385 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770294492.122126  800385 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770294492.122128  800385 computation_placer.cc:177] computation placer alr

In [2]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatTransformerReconstructor(data_config, name="Transformer")

In [3]:
Transformer.build_model(
    hidden_dim=64,
    num_layers=6,
    dropout_rate=0.1,
)


I0000 00:00:1770294604.388799  800385 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13191 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [ ]:
Transformer.adapt_normalization_layers(X_train)
Transformer.compile_model(
    loss={
        "assignment": utils.AssignmentLoss(),
        "normalized_regression": utils.PtEtaPhiLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [utils.AssignmentAccuracy(name="accuracy")],
        "normalized_regression": [utils.RegressionDeviation(name="deviation")],
    },
    #add_physics_informed_loss=True,
    loss_weights={"assignment": 1.0, "normalized_regression": 1.0}
)

I0000 00:00:1770294607.783804  802308 service.cc:152] XLA service 0x7ff49c003a90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770294607.783829  802308 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2026-02-05 13:30:07.808428: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1770294607.861350  802308 cuda_dnn.cc:529] Loaded cuDNN version 91500
I0000 00:00:1770294608.143342  802308 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Adapted normalization layer:  hlf_input_normalization
Adapted normalization layer:  met_input_normalization
Adapted normalization layer:  lep_input_normalization
Adapted normalization layer:  jet_input_normalization
Set regression denormalization layer with computed mean and std.


In [5]:
Transformer.train_model(
    epochs=10,
    data=X_train,
    sample_weights=utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
    validation_split=0.1,
)

Epoch 1/10
3511/3512 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - assignment_assignment_accuracy: 0.2040 - assignment_loss: 0.1738 - loss: 5.9658 - normalized_regression_loss: 5.7920 - normalized_regression_regression_deviation: 0.0760

2026-02-05 13:33:56.001822: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_10', 124 bytes spill stores, 104 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_12', 112 bytes spill stores, 92 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_13', 112 bytes spill stores, 92 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_2', 16 bytes spill stores, 16 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_9', 16 bytes spill stores, 16 bytes spill loads



3512/3512 ━━━━━━━━━━━━━━━━━━━━ 222s 43ms/step - assignment_assignment_accuracy: 0.2040 - assignment_loss: 0.1738 - loss: 5.9655 - normalized_regression_loss: 5.7917 - normalized_regression_regression_deviation: 0.0760 - val_assignment_assignment_accuracy: 0.0178 - val_assignment_loss: 0.1697 - val_loss: 5.5086 - val_normalized_regression_loss: 5.3394 - val_normalized_regression_regression_deviation: 0.0577 - learning_rate: 1.0000e-04
Epoch 2/10
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 113s 32ms/step - assignment_assignment_accuracy: 0.0557 - assignment_loss: 0.1767 - loss: 5.5025 - normalized_regression_loss: 5.3258 - normalized_regression_regression_deviation: 0.0577 - val_assignment_assignment_accuracy: 0.1170 - val_assignment_loss: 0.1698 - val_loss: 5.7692 - val_normalized_regression_loss: 5.6001 - val_normalized_regression_regression_deviation: 0.0588 - learning_rate: 1.0000e-04
Epoch 3/10
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 114s 32ms/step - assignment_assignment_accuracy: 0.1153 - assignment_lo

In [6]:
Transformer.save_model(MODEL_DIR + "odd_model.keras")

Model saved to models/regression_transformer/odd_model.keras


In [7]:
upscale_layer = Transformer.model.get_layer("regression")

In [8]:
import numpy as np
import tensorflow as tf
np.abs(upscale_layer(y_train["regression"] / y_train["regression"].std(axis = 0)) - X_train["regression"]).mean()

np.float32(640016.2)

In [9]:
from core.components import reco_W_mass_deviation
leptons = X_train["lep_inputs"]
neutrino_pred = Transformer.reconstruct_neutrinos(X_train)
neutrino_true = X_train["regression"]
neutrino_nu_flows = X_train["nu_flows_neutrino_truth"]

In [10]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_pred, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

np.float32(0.27241772)

In [11]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_true, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

np.float32(0.0065785283)

In [12]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_nu_flows, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

np.float32(0.096865565)